In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-100rfq-30cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

7690

In [5]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

accept_count = 0
accept_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'ACCEPT':
        accept_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        accept_count += 1   


print(f"#ACCEPTs =  {accept_count}")

#ACCEPTs =  100


In [12]:
df_test = grouped_data.get_group("331ca77987ad25091c086a5a8534dc766be3c41bcc3803fb829a498f0a7a168e")
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
15838,received_tx,106,ACCEPT,331ca77987ad25091c086a5a8534dc766be3c41bcc3803...,None,1.669859e+09
15840,before_tendermint,118,ACCEPT,331ca77987ad25091c086a5a8534dc766be3c41bcc3803...,None,1.669859e+09
15842,check_tx,131,ACCEPT,331ca77987ad25091c086a5a8534dc766be3c41bcc3803...,None,1.669859e+09
15844,deliver_tx,150,ACCEPT,331ca77987ad25091c086a5a8534dc766be3c41bcc3803...,None,1.669859e+09
15845,end_block,152,ACCEPT,331ca77987ad25091c086a5a8534dc766be3c41bcc3803...,None,1.669859e+09
15846,commit_tx,286,ACCEPT,331ca77987ad25091c086a5a8534dc766be3c41bcc3803...,None,1.669859e+09


## without RETURN 

In [25]:
time_all_committed_accepts = {}
time_passed_accepts = []
validated_time = []
accepted_time = []
committed_time = []

accept_underfit = []
accept_underfit_dict = {}

accept_overfit = []
accept_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            accept_underfit.append(temp_list)
            accept_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_accepts[tx_id[i]] = actual_time
            time_passed_accepts.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            accept_overfit.append(actual_time)
            accept_overfit_dict[tx_id[i]] = temp_list

## with RETURNs

In [22]:


accept_return_count = {}

time_id_return = {}
for i in range(0, len(tx_id)):
    temp_list = []
    return_count = 0
    if "ACCEPT" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        for j in range(0, len(data)):
            if data['Event'][j] == "commit_tx" and tx_id[i] == data['Accept_id'][j]:
                temp_list.append(data["Time"][j])
                return_count += 1
        accept_return_count[tx_id[i]] = return_count
        

In [ ]:
accept_return_count

In [ ]:
time_all_committed_accepts

In [28]:
len(time_passed_accepts)

100

In [18]:
len(accept_overfit)

92

In [26]:
stat.mean(time_passed_accepts)

344.57

In [27]:
stat.mean(accept_overfit)

356.9021739130435

In [31]:
# Analysis of CREATE Transaction
accept_mean = "{:.2f}".format(stat.mean(time_passed_accepts))
accept_median = stat.median(time_passed_accepts)
accept_max = max(time_passed_accepts)
accept_min = min(time_passed_accepts)
accept_std_dev = "{:.2f}".format(stat.stdev(time_passed_accepts))

print(accept_mean)
print(accept_median)
print(accept_max)
print(accept_min)
print(accept_std_dev)
print(f"total accepts = {len(time_passed_accepts)}")

accept_dict = {
               'min': accept_min,
               'max': accept_max,
               'average': accept_mean,
               'median' : accept_median,
               'standard_deviation' : accept_std_dev,
               '# of txs' : len(time_passed_accepts)}

344.57
354.0
408
178
45.09
total accepts = 100


In [32]:
results = pd.DataFrame(accept_dict, index = ['ACCEPT'])
results.to_csv('accept_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
ACCEPT,178,408,344.57,354.0,45.09,100
